In [3]:
from dataclasses import dataclass

@dataclass
class Packet:
    name: str
    flow: str      # "top", "mid", "bot"
    arrival: float
    length: float
    weight: float  # flow weight
    S: float = 0.0
    F: float = 0.0

def wfq_order(packets):
    # Tracks last finish per flow
    lastF = {}
    out = []

    # Processes in **per-flow** arrival order
    for p in sorted(packets, key=lambda x: (x.flow, x.arrival, x.name)):
        prevF = lastF.get(p.flow, 0.0)
        V = p.arrival
        p.S = max(prevF, V)
        p.F = p.S + (p.length / p.weight)
        lastF[p.flow] = p.F
        out.append(p)

    # Transmission order: sorts by (finish time, name)
    out.sort(key=lambda x: (x.F, x.name))
    return [p.name for p in out], out

if __name__ == "__main__":
    # Fig. 5-29(a): bottom queue has weight 2; top & middle have weight 1
    W = {"top": 1.0, "mid": 1.0, "bot": 2.0}

    packets = [
        # top
        Packet("A", "top", 0, 8,  W["top"]),
        Packet("F", "top",10, 6,  W["top"]),
        # middle
        Packet("B", "mid", 5, 6,  W["mid"]),
        Packet("D", "mid", 8, 9,  W["mid"]),
        Packet("H", "mid",20, 8,  W["mid"]),
        # bottom (weight 2x)
        Packet("C", "bot", 5,10,  W["bot"]),
        Packet("E", "bot", 8, 8,  W["bot"]),
        Packet("G", "bot",11,10,  W["bot"]),
    ]

    order, tagged = wfq_order(packets)
    print("Output Order:", " ".join(order))

    for p in sorted(tagged, key=lambda x: (x.F, x.name)):
        print(f"{p.name}: S={p.S:g}, F={p.F:g}")

Output Order: A C B E F G D H
A: S=0, F=8
C: S=5, F=10
B: S=5, F=11
E: S=10, F=14
F: S=10, F=16
G: S=14, F=19
D: S=11, F=20
H: S=20, F=28
